# Mediapipe

In [1]:
import cv2
import mediapipe as mp
import openpifpaf
import PIL
import torch
from time import time

2023-06-02 17:26:46.489977: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-02 17:26:47.629014: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/guillaume/anaconda3/envs/semproj/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda-11.7/lib64:
2023-06-02 17:26:47.629084: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/guillaume/ana

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
NVIDIA GeForce RTX 3050 Laptop GPU


In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic

In [5]:
# For static images:
IMAGE_FILES = []
BG_COLOR = (192, 192, 192) # gray
with mp_holistic.Holistic(
    static_image_mode=True,
    model_complexity=2,
    enable_segmentation=True,
    refine_face_landmarks=True) as holistic:
  for idx, file in enumerate(IMAGE_FILES):
    image = cv2.imread(file)
    image_height, image_width, _ = image.shape
    # Convert the BGR image to RGB before processing.
    results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if results.pose_landmarks:
      print(
          f'Nose coordinates: ('
          f'{results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x * image_width}, '
          f'{results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y * image_height})'
      )

    annotated_image = image.copy()
    # Draw segmentation on the image.
    # To improve segmentation around boundaries, consider applying a joint
    # bilateral filter to "results.segmentation_mask" with "image".
    condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
    bg_image = np.zeros(image.shape, dtype=np.uint8)
    bg_image[:] = BG_COLOR
    annotated_image = np.where(condition, annotated_image, bg_image)
    # Draw pose, left and right hands, and face landmarks on the image.
    mp_drawing.draw_landmarks(
        annotated_image,
        results.face_landmarks,
        mp_holistic.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_tesselation_style())
    mp_drawing.draw_landmarks(
        annotated_image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.
        get_default_pose_landmarks_style())
    cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)
    # Plot pose world landmarks.
    mp_drawing.plot_landmarks(
        results.pose_world_landmarks, mp_holistic.POSE_CONNECTIONS)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# Using the webcam

In [12]:
predictor = openpifpaf.Predictor(checkpoint='shufflenetv2k16')

pos_dict = {
        "nose": 0,
        "left_eye": 1,
        "right_eye": 2,
        "left_ear": 3,
        "right_ear": 4,
        "left_shoudler": 5,
        "right_shoudler": 6,
        "left_elbow": 7,
        "right_elbow": 8,
        "left_wrist": 9,
        "right_wrist": 10,
        "left_hip": 11,
        "right_hip": 12,
        "left_knee": 13,
        "right_knee": 14,
        "left_ankle": 15,
        "right_ankle": 16}

RED = (0, 0, 255)
GREEN = (0, 255, 0)
BLUE = (255, 0, 0)
YELLOW = (0, 255, 255)

In [8]:
OPPF = False

# For webcam input:
cap = cv2.VideoCapture()
with mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    start = time()
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'co£ntinue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    if OPPF:
        #OpenPifPaf
        pil_img = PIL.Image.fromarray(image)
        predictions, _, _ = predictor.pil_image(pil_img)

        try:
            
            #draw arms and hips
            keypoints = predictions[0].data

            # left arm
            left_shoulder = keypoints[pos_dict["left_shoudler"],0:2]
            left_hip = keypoints[pos_dict["left_hip"],0:2]
            left_elbow = keypoints[pos_dict["left_elbow"],0:2]

            # right arm
            right_shoulder = keypoints[pos_dict["right_shoudler"],0:2]
            right_hip = keypoints[pos_dict["right_hip"],0:2]
            right_elbow = keypoints[pos_dict["right_elbow"],0:2]

            cv2.line(image, tuple(map(int, tuple(left_shoulder))), tuple(map(int, tuple(left_elbow))), RED, 5)
            cv2.line(image, tuple(map(int, tuple(left_shoulder))), tuple(map(int, tuple(left_hip))), YELLOW, 5)
            cv2.line(image, tuple(map(int, tuple(right_shoulder))), tuple(map(int, tuple(right_elbow))), RED, 5)
            cv2.line(image, tuple(map(int, tuple(right_shoulder))), tuple(map(int, tuple(right_hip))), YELLOW, 5)
        except:
            cv2.putText(img = image, text=f"no detection", org = (0,60), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=GREEN,thickness=2)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Holistic', cv2.flip(image, 1))
    c = cv2.waitKey(1)
    if c == 27: # press escape to quit
        break
    stop = time()

    # writes fps
    fps = 1/(stop-start)
    cv2.putText(img = image, text=f"{fps:.2f} fps", org = (0,30), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=5, color=GREEN,thickness=2)

cap.release()
cv2.destroyAllWindows()


In [10]:
import cv2
import mediapipe as mp
import numpy as np
import time 
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

new_frame_time = 0
prev_frame_time = 0

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    new_frame_time = time.time()
    fps = 1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    print(image.shape)
    results = pose.process(image)

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.
    fps = str(int(fps))
    cv2.putText(image, fps, (7, 70), 1, 3, (100, 255, 0), 3, cv2.LINE_AA)
    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    c = cv2.waitKey(1)
    if c == 27: # press escape to quit
        break
cap.release()

[ WARN:0@206.808] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@206.926] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


0 - nose
1 - left eye (inner)
2 - left eye
3 - left eye (outer)
4 - right eye (inner)
5 - right eye
6 - right eye (outer)
7 - left ear
8 - right ear
9 - mouth (left)
10 - mouth (right)
11 - left shoulder
12 - right shoulder
13 - left elbow
14 - right elbow
15 - left wrist
16 - right wrist
17 - left pinky
18 - right pinky
19 - left index
20 - right index
21 - left thumb
22 - right thumb
23 - left hip
24 - right hip
25 - left knee
26 - right knee
27 - left ankle
28 - right ankle
29 - left heel
30 - right heel
31 - left foot index
32 - right foot index


In [32]:
name_to_index = {"nose":0,
"left eye (inner)":1,
"left eye":2,
"left eye (outer)":3,
"right eye (inner)":4,
"right eye":5,
"right eye (outer)":6,
"left ear":7,
"right ear":8,
"mouth (left)":9,
"mouth (right)":10,
"left shoulder":11,
"right shoulder":12,
"left elbow":13,
"right elbow":14,
"left wrist":15,
"right wrist":16,
"left pinky":17,
"right pinky":18,
"left index":19,
"right index":20,
"left thumb":21,
"right thumb":22,
"left hip":23,
"right hip":24,
"left knee":25,
"right knee":26,
"left ankle":27,
"right ankle":28,
"left heel":29,
"right heel":30,
"left foot index":31,
"right foot index":32}


In [ ]:
img_size = [480,640]

In [28]:
results.pose_landmarks.landmark[0]

x: 0.6065418720245361
y: 0.5476277470588684
z: -2.819145917892456
visibility: 0.998938798904419

In [29]:
results.pose_landmarks.landmark

[x: 0.6065418720245361
y: 0.5476277470588684
z: -2.819145917892456
visibility: 0.998938798904419
, x: 0.6495431661605835
y: 0.4248652160167694
z: -2.751957416534424
visibility: 0.9977502226829529
, x: 0.6757246255874634
y: 0.42076441645622253
z: -2.752040147781372
visibility: 0.9980242252349854
, x: 0.702030599117279
y: 0.4170161187648773
z: -2.7525336742401123
visibility: 0.9970047473907471
, x: 0.5584686398506165
y: 0.4247419238090515
z: -2.774761199951172
visibility: 0.9981383681297302
, x: 0.5218402743339539
y: 0.4202928841114044
z: -2.7738561630249023
visibility: 0.9985030889511108
, x: 0.48931750655174255
y: 0.4157469570636749
z: -2.7744686603546143
visibility: 0.9982019662857056
, x: 0.7390888929367065
y: 0.4269128441810608
z: -1.9621719121932983
visibility: 0.997580349445343
, x: 0.4308583736419678
y: 0.42057880759239197
z: -2.0330443382263184
visibility: 0.9991651773452759
, x: 0.6449480056762695
y: 0.6446468830108643
z: -2.487389087677002
visibility: 0.9988939762115479
, x: 0